In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, train_test_split
from hyperopt import tpe
import numpy as np

In [3]:
df = pd.read_csv('clean_data.csv')

In [4]:
pdf = df.reindex(np.random.permutation(df.index))

In [5]:
X, y = pdf['clean_text'], pdf['class']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y[:, np.newaxis], test_size=.33, random_state=42)

In [7]:
vec = CountVectorizer(ngram_range=(1, 3), max_features=100)
X_train_enc = vec.fit_transform(X_train)
X_test_enc = vec.transform(X_test)
X_enc = vec.fit_transform(X)

In [8]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
def loss(y_target, y_pred):
    acc = accuracy_score(y_target, y_pred)
    print(f"this loss called with acc={acc}")
    return 1.0 - acc

In [11]:
import xgboost

In [22]:
from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing, any_sparse_classifier, sgd, passive_aggressive, svc_linear, decision_tree, xgboost_classification, liblinear_svc
import hyperopt
from hyperopt import tpe
from hyperopt import hp
np.random.seed(13)
#sgd('sgd'), svc_linear('svc'), 
estim = HyperoptEstimator(classifier=hp.choice('cls', [liblinear_svc('svc')]),
                         preprocessing=[], max_evals=100, algo=tpe.suggest, trial_timeout=10)
estim.fit(X_enc, y, cv_shuffle=True)

100%|██████████| 1/1 [00:00<00:00, 13.79it/s, best loss: 0.47250000000000003]


In [25]:
estim.best_model()

{'learner': LinearSVC(C=988.8915413729198, class_weight=None, dual=True, fit_intercept=True,
           intercept_scaling=0.5255498355851986, loss='squared_hinge',
           max_iter=1000, multi_class='ovr', penalty='l2', random_state=1,
           tol=0.003219411190972894, verbose=False),
 'preprocs': (),
 'ex_preprocs': ()}

In [26]:
cross_val_score(estim.best_model()['learner'], X_enc, y, cv=5).mean()

0.6415

In [17]:
cross_val_score(SGDClassifier(random_state=41), X_enc, y, cv=5).mean()

0.6695